In [40]:
from collections import defaultdict
from array import array
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import random
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\polam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\polam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
data_folder = './data/'
output_folder = './output/'

data_path = data_folder + 'tw_hurricane_data.json'
map_path = data_folder + 'tweet_document_ids_map.csv'


with open(data_path) as file:
    lines = file.readlines()
    
print("Total number of tweets read: {}".format(len(lines)))

lines = [json.loads(line) for line in lines]

Total number of tweets read: 4000


In [12]:
def clean(text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    text = text.lower()
    text = word_tokenize(text)
    text = [token for token in text if token.isalpha()]
    text = [x for x in text if x not in stop_words]  
    text = [stemmer.stem(x) for x in text]
    
    return text

In [44]:
def feature_extraction(tweet_line):
    tweet = {}
    tweet['text'] = tweet_line['full_text']
    tweet['username'] = tweet_line['user']['screen_name']
    tweet['date'] = tweet_line['created_at']
    tweet['hashtags'] = [x['text'] for x in tweet_line['entities']['hashtags']]
    tweet['mentions'] = re.findall("@([a-zA-Z0-9_]{1,50})", tweet['text'])
    tweet['likes'] = tweet_line['favorite_count']
    tweet['retweets'] = tweet_line['retweet_count']
    tweet['url'] = 'https://twitter.com/'+tweet['username']+'/status/'+tweet_line['id_str']
    tweet['tk_text'] = clean(tweet_line['full_text'])
    
    return tweet

In [59]:
l = random.randint(0, len(lines)-1)
new_tweet = feature_extraction(lines[l])

print('Default tweet: \t{}' + repr(new_tweet['text']))
print('Tweet: \t\t{}'.format(new_tweet['tk_text']))
print('Username: \t{}'.format(new_tweet['username']))
print('Date: \t\t{}'.format(new_tweet['date']))
print('Hashtags: \t{}'.format(new_tweet['hashtags']))
print('Mentions: \t{}'.format(new_tweet['mentions']))
print('Likes: \t\t{}'.format(new_tweet['likes']))
print('Retweets: \t{}'.format(new_tweet['retweets']))
print('Url: \t\t{}\n'.format(new_tweet['url']))


Default tweet: 	{}'. @GovRonDeSantis is requesting quick insurance claim settlements for Floridians. \n\nWe don’t need the National Flood Insurance Program (NFIP) fighting w/ private insurance carriers over wind vs. flood damage. I will be watching this closely. \n\nPray for our state.  #HurricaneIan'
Tweet: 		['govrondesanti', 'request', 'quick', 'insur', 'claim', 'settlement', 'floridian', 'need', 'nation', 'flood', 'insur', 'program', 'nfip', 'fight', 'privat', 'insur', 'carrier', 'wind', 'flood', 'damag', 'watch', 'close', 'pray', 'state', 'hurricaneian']
Username: 	CoryMillsFL
Date: 		Fri Sep 30 14:46:08 +0000 2022
Hashtags: 	['HurricaneIan']
Mentions: 	['GovRonDeSantis']
Likes: 		315
Retweets: 	85
Url: 		https://twitter.com/CoryMillsFL/status/1575859547012534273

